Importing necessary modules

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

The dataset is not large enough so it's nit suitable for such a complicated task. We will download the same tickers of the original dataset but beginning from the first available date.

In [2]:
start_date = "2000-08-24" #First date where information is available
end_date = "2011-08-05" #last day in the original training set
# List of tickers to fetch data for
tickers = ['CL=F','EURUSD=X','GC=F','HG=F','JPY=X','SI=F','XWD.TO','^DJI','^GSPC','^IXIC','^RUT','^TNX','^TYX','^VIX']

# Fetch data for each ticker and store in a dictionary
Series = {ticker:yf.download(ticker, start=start_date, end=end_date)['Close'] for ticker in tickers}
data=pd.DataFrame(Series) #building the final dataframe

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
data.head()

,CL=F,EURUSD=X,GC=F,HG=F,JPY=X,SI=F,XWD.TO,^DJI,^GSPC,^IXIC,^RUT,^TNX,^TYX,^VIX
Date,,,,,,,,,,,,,,
2000-08-24,31.629999,NaN,NaN,NaN,106.989998,NaN,NaN,11182.740234,1508.310059,4053.280029,523.299988,5.716,5.659,17.040001
2000-08-25,32.049999,NaN,NaN,NaN,106.800003,NaN,NaN,11192.629883,1506.449951,4042.679932,525.109985,5.721,5.668,16.530001
2000-08-28,32.869999,NaN,NaN,NaN,106.589996,NaN,NaN,11252.839844,1514.089966,4070.590088,526.479980,5.766,5.715,16.540001
2000-08-29,32.720001,NaN,NaN,NaN,106.129997,NaN,NaN,11215.099609,1509.839966,4082.169922,529.630005,5.808,5.751,16.889999
2000-08-30,33.400002,NaN,273.899994,0.885,106.610001,4.93,NaN,11103.009766,1502.589966,4103.810059,532.330017,5.800,5.736,17.690001


We still need to calculate the target from the ^GSPC ticker.

In [4]:
data['^GSPC'].isna().sum() # number of missing values in the GSPC column

103

Since the number of missing values is not important compared to the total number of rows, we decided to drop the rows where values from GSPC are missing.

In [6]:
data.dropna(subset=['^GSPC'],inplace=True) #dropping the missing values

Calculating the target

In [8]:
data['^GSPC Shifted'] = data['^GSPC'].shift(-1) #Creating a shifted column that will serve as the future closing price
data['target'] = (data['^GSPC'] < data['^GSPC Shifted']).astype(int) #computing the target (0 if the price will go up and 1 if it will go down)
data.drop(columns=['^GSPC Shifted'],inplace=True) #dropping the unnecessary column

Let's look at the missing values in other columns.

In [9]:
data.isna().sum()

CL=F          13
EURUSD=X     843
GC=F          20
HG=F          19
JPY=X         25
SI=F          19
XWD.TO      2319
^DJI           0
^GSPC          0
^IXIC          0
^RUT           0
^TNX           5
^TYX           5
^VIX           0
target         0
dtype: int64

We need to find a method to impute these missing values. We chose to use panda's interpolate function with the method 'time' because it takes into account the time evolution.

In [10]:
#Converting the Date column to datetime to ensure the interpolate method works properly
data.reset_index(inplace=True)
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date',inplace=True)

In [11]:
new_data=data.interpolate(method='time') #calling the interpolate method

Let's look at the missing values in the new data set.

In [12]:
new_data.isna().sum()

CL=F           0
EURUSD=X     819
GC=F           4
HG=F           4
JPY=X          0
SI=F           4
XWD.TO      2308
^DJI           0
^GSPC          0
^IXIC          0
^RUT           0
^TNX           0
^TYX           0
^VIX           0
target         0
dtype: int64

Turns out the interpolate method leaves the points where no neighboring points are available. It's clearly the case for tickers EURUSD=X and XWD.TO. We decided to drop them since imputing with another method would give us low quality data.

In [67]:
# Dropping columns where too many values are missing
del new_data['EURUSD=X']
del new_data['XWD.TO']

In [68]:
new_data.dropna(inplace=True) #dropping other missing values

Saving the new dataset in a csv file

In [69]:
new_data.to_csv('full_train.csv')